<a href="https://colab.research.google.com/github/dasd412/my-first-ai/blob/main/logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

fish=pd.read_csv('https://bit.ly/fish_csv_data')
fish.head()

In [ ]:
# 어떤 종류의 생선이 있는지 species 열에서 고유한 값 추출해보기
print(pd.unique(fish['Species']))

In [ ]:
# species열만 타깃 데이터로 만들기 위해 나머지 5개 열은 입력 데이터로 사용
fish_input=fish[['Weight','Length','Diagonal','Height','Width']].to_numpy()

In [ ]:
print(fish_input[:5])

In [ ]:
# 타깃 데이터 만들기
fish_target=fish['Species'].to_numpy()

In [ ]:
from sklearn.model_selection import train_test_split

train_input,test_input,train_target,test_target=train_test_split(fish_input,fish_target,random_state=42) # 훈련 세트와 테스트 세트의 타깃 데이터 모두에 7개의 생선 종류가 들어가 있다.
# 이렇게 타깃 데이터에 2개 이상의 클래스가 포함된 문제를 다중 분류라고 부른다.

In [ ]:
# 훈련 세트와 테스트 세트 표준화 전처리
from sklearn.preprocessing import StandardScaler

ss=StandardScaler()
ss.fit(train_input)
# 훈련 세트의 통계 값으로 테스트 세트를 변환해야 한다!!
train_scaled=ss.transform(train_input)
test_scaled=ss.transform(test_input)


In [ ]:
# knn 분류기로 테스트 세트에 들어 있는 확률을 예측해보자.
from sklearn.neighbors import KNeighborsClassifier

kn=KNeighborsClassifier(n_neighbors=3)
kn.fit(train_scaled,train_target)
print(kn.score(train_scaled,train_target))
print(kn.score(test_scaled,test_target))

In [ ]:
print(kn.classes_)

In [ ]:
print(kn.predict(test_scaled[:5]))

In [ ]:
import numpy as np
proba=kn.predict_proba(test_scaled[:5])
print(np.round(proba,decimals=4))

In [ ]:
distances,indexes=kn.kneighbors(test_scaled[3:4])
print(train_target[indexes])

In [ ]:
# 로지스틱 회귀는 이름은 회귀이지만, 분류 모델이다. 선형 회귀와 동일하게 선형 방정식을 학습한다.
# 시그모이드 함수를 사용하면 0~1의 값으로 나타낼 수 있게 된다.
import numpy as np
import matplotlib.pyplot as plt

z=np.arange(-5,5,0.1) # -5 ~ 5 사이에 0.1 간격으로 배열 z를 만듬
phi =1/(1+np.exp(-z)) # z위치마다 시그모이드 함수 계산
plt.plot(z,phi)
plt.xlabel('z')
plt.ylabel('phi')
plt.show()

In [ ]:
# 넘파이 배열은 True,False 값을 전달하여 행을 선택할 수 있는데, 이를 불리언 인덱싱이라고 한다.
# 다음은 A와 C만 골라내는 예제이다.

char_arr=np.array(['A','B','C','D','E'])
print(char_arr[[True,False,True,False,False]])

In [ ]:
bream_smelt_indexes=(train_target=='Bream') | (train_target=='Smelt')
train_bream_smelt=train_scaled[bream_smelt_indexes]
target_bream_smelt=train_target[bream_smelt_indexes]

In [ ]:
#로지스틱 회귀로 이진 분류를 수행해보자.
from sklearn.linear_model import LogisticRegression

lr=LogisticRegression()
lr.fit(train_bream_smelt,target_bream_smelt)

In [ ]:
print(lr.predict(train_bream_smelt[:5]))

In [ ]:
# 예측 확률 , 첫 번째 열이 음성 클래스 (0)에 대한 확률이고 두 번째 열이 양성 클래스 (1)에 대한 확률.
print(lr.predict_proba(train_bream_smelt[:5]))

In [ ]:
#classes_ 속성을 이용해 타깃 데이터의 순서를 알 수 있음. 음성 클래스가 Bream, 양성 클래스가 Smelt
print(lr.classes_)

In [ ]:
# 로지스틱 회귀가 학습한 계수 확인해보기
print(lr.coef_,lr.intercept_)

In [ ]:
# 로지스틱 회귀 결괏값 확인 (단, 양성 클래스에 한해서만 값을 반환함.)
decisions=lr.decision_function(train_bream_smelt[:5])
print(decisions)

In [ ]:
# 위 결괏값을 확률로 표현
from scipy.special import expit
print(expit(decisions)) # expit()은 시그모이드 함수와 동일함.

In [ ]:
#로지스틱 회귀로 다중 분류를 해보자!

# LogisticRegression 은 기본적으로 릿지 회귀와 같이 계수의 제곱을 규제한다. 이런 규제를 L2 규제라고 한다.
# 그리고 규제를 제어하는 변수를 C라 한다. C가 작을수록 규제가 커진다.

lr=LogisticRegression(C=20,max_iter=1000)#max_iter는 반복 횟수이다. LogisticRegression은 기본적으로 반복적인 알고리즘을 사용한다.
lr.fit(train_scaled,train_target)
print(lr.score(train_scaled, train_target))
print(lr.score(test_scaled,test_target))

In [ ]:
print(lr.predict(test_scaled[:5]))

In [ ]:
# 예측 확률 출력, 5개 샘플 예측이므로 5개의 행이 출력되었다. 그리고 7개 생선에 대한 확률이므로 7개의 열이 출력됬다.
proba=lr.predict_proba(test_scaled[:5])
print(np.round(proba,decimals=3))

In [ ]:
print(lr.coef_.shape,lr.intercept_.shape) # 다중 분류에서는 시그모이드 함수가 아니라 소프트맥스 함수를 사용한다.

In [ ]:
decision=lr.decision_function(test_scaled[:5])
print(np.round(decision,decimals=2))

In [ ]:
from scipy.special import softmax
proba=softmax(decision,axis=1)# axis 매개변수는 소프트맥스를 계산할 축을 지정한다.axis=1이면, 각행에 대해 소프트맥스를 계산함.
print(np.round(proba,decimals=3))